In [19]:
cd /home/drew/NAP/traj

/scr/drew/Serotonin/ligands/25N-1-NAP/6WHA/noGProtein/IFD_1/play/tryIonicCapsInSolvent/traj


In [20]:
with open("castp_raw.txt") as f:
    job_ids = []
    for line in f:
        cleanline = line.strip()
        if not cleanline.startswith("echo"):
            continue
        _,pair = cleanline.split(":")
        _,job_id = pair.split(",")
        job_ids.append(f'"{job_id}"')

In [21]:
f"[{','.join(job_ids)}]"

'["j_6233b66ed4da1","j_6233b6804548a","j_6233b691ace66","j_6233b6a314ff8","j_6233b6b4659a6","j_6233b6c5af1f4","j_6233b6d70f18f","j_6233b6e874591","j_6233b6f9cd763","j_6233b70b234a3","j_6233b71c71730","j_6233b72dc04b4","j_6233b73f153f7","j_6233b7506dcb2","j_6233b761ceb1b","j_6233b7732e7f3","j_6233b7848a5a0","j_6233b795d9d42","j_6233b7a72e358","j_6233b7b889df7","j_6233b7c9dc68b","j_6233b7db315ea","j_6233b7ec816c2","j_6233b7fdcfe32","j_6233b80f2a62f","j_6233b820772c0","j_6233b831c9d62","j_6233b84328b07","j_6233b8547b09f","j_6233b865db1ad","j_6233b87753354","j_6233b888bb20a","j_6233b89a158ba","j_6233b8ab8144d","j_6233b8bce060a","j_6233b8ce4beb2","j_6233b8dface59","j_6233b8f10de8c","j_6233b9026c80c","j_6233b913c1ec9","j_6233b9253031f","j_6233b93689b62","j_6233b94804e9a","j_6233b9596641c","j_6233b96ad28e7","j_6233b97c524ab","j_6233b98dbfbbc","j_6233b99f45797","j_6233b9b0b1c9b","j_6233b9c213a89","j_6233b9d37ea29","j_6233b9e4f3c5c","j_6233b9f6629b5","j_6233ba07b6791","j_6233ba19855fe","j_6233b

In [22]:
import csv
import bioinf as bi

with open("castp_analysis_raw.txt") as f:
    ic_volumes = []
    ec_volumes = []
    t = 0
    for line in f:
        cleanline = line.strip()
        if not cleanline.startswith("echo"):
            continue
        _, values = cleanline.split(":")
        large_volume, large_resSeq, small_volume, small_resSeq = values.split(",")
        resSeqs = (large_resSeq, small_resSeq)
        with open(f"everynsProt{t}.pdb") as p:
            zs = {}
            for line in p:
                cleanline = line.strip()
                if not line.startswith("ATOM"):
                    continue
                pdbline = bi.parse_pdb_ATOM_line(cleanline)
                resSeq = pdbline["resSeq"]
                if not resSeq in resSeqs:
                    continue
                if pdbline["name"] != "CA":
                    continue
                zs[resSeq] = float(pdbline["z"])
        ec_is_smaller = zs[small_resSeq] < zs[large_resSeq]
        ic_volumes.append(large_volume if ec_is_smaller else small_volume)
        ec_volumes.append(small_volume if ec_is_smaller else large_volume)
        small_volumes.append(small_volume)
        t += 1
with open("ecVolume.dat", 'w') as ec, open("icVolume.dat", 'w') as ic:
    for i,volume in enumerate(ic_volumes):
        ic.write(f"{volume}\n")
        ec.write(f"{ec_volumes[i]}\n")
